Please load the Planet.csv file to execute both visualisation.

In [1]:
# FIRST VISUALISATION: (Surface Gravity of Each Solar System Planet)

import pandas as pd
import plotly.express as px
# Reading the CSV file into 2 DataFrame
df = pd.read_csv('Planet.csv', encoding='iso-8859-1')
df1 = pd.read_csv('Planet.csv', encoding='iso-8859-1')

# Set the volume and mass conversion to scientific notation
# converts Volume(cm3) column to string
df["Volume(km3)"] = df["Volume(km3)"].astype(str)
# removing commas from Volume(cm3) column
df["Volume(km3)"] = df["Volume(km3)"].str.replace(',', '')
# Convert Volume(cm3) column to numeric and handle non-numeric values
df["Volume(km3)"] = pd.to_numeric(df["Volume(km3)"], errors='coerce')
# convert Mass(kg) column to string
df["Mass(kg)"] = df["Mass(kg)"].astype(str)
# remove commas from Mass(kg)) column
df["Mass(kg)"] = df["Mass(kg)"].str.replace(',', '')
# Convert Mass(kg) column to numeric and handle non-numeric values
df["Mass(kg)"] = pd.to_numeric(df['Mass(kg)'], errors='coerce')

# Drop any rows with null values
df = df.dropna(subset=["Volume(km3)"])
# Reshape DataFrame to long-form
df2 = df.melt(id_vars="Planet", value_vars=["Mass(kg)"], var_name="Mass", value_name="Value_mass")
df = df.melt(id_vars="Planet", value_vars=["Volume(km3)"], var_name="Metric", value_name="Value_volume")
# Convert Volume data to scientific notation with custom format
df["Volume_Scientific"] = df["Value_volume"].apply(lambda x: "{:.2e}".format(x))
df["Mantissa_volume"] = df["Volume_Scientific"].apply(lambda x: float(x.split("e")[0]))
df["Exponent_volume"] = df["Volume_Scientific"].apply(lambda x: int(x.split("e")[1]))
df["Mantissa_Formatted_volume"] = df["Mantissa_volume"].apply(lambda x: "{:.2f}".format(x))
# Convert Mass data to scientific notation with custom format
df2["Mass_Scientific"] = df2["Value_mass"].apply(lambda x: "{:.2e}".format(x))
df2["Mantissa_mass"] = df2["Mass_Scientific"].apply(lambda x: float(x.split("e")[0]))
df2["Exponent_mass"] = df2["Mass_Scientific"].apply(lambda x: int(x.split("e")[1]))
df2["Mantissa_Formatted_mass"] = df2["Mantissa_mass"].apply(lambda x: "{:.2f}".format(x))

# Selecting color to be displayed on each planet
color_map = {
    'Mercury': '#A9A9A9',     # Silver Chalice
    'Venus': '#FFD700',       # Gold
    'Earth': '#4682B4',       # Steel Blue
    'Mars': '#CD5C5C',        # Chestnut Rose
    'Jupiter': '#FF6347',     # Persimmon
    'Saturn': '#ffdea7',      # Navajo White
    'Uranus': '#87CEEB',      # Seagull
    'Neptune': '#3d5ef9'      # Dodger Blue
}

# Merge mass data from df2 into df
df = pd.merge(df, df2, on="Planet", suffixes=('_volume', '_mass'))
# Create the scatter plot
fig = px.scatter(df,
                 x=df1['Orbit Distance(km)'],  # Use the 'Orbit Distance(km)' column for x-axis values
                 y=df1['Surface Gravity(m/s2)'],
                 size="Value_volume",
                 size_max=150,
                 hover_data={"Planet": True, "Value_volume": True, "Mantissa_Formatted_volume": True, "Exponent_volume": True, "Value_mass": True},
                 custom_data=["Mantissa_Formatted_volume", "Exponent_volume", "Mantissa_Formatted_mass", "Exponent_mass",df1["Orbit Distance(km)"]],
                 color="Planet",
                 color_discrete_map=color_map,
                 labels={'Value': 'Volume (cm3)'},
                 title='Surface Gravity(m/s2) of Solar System Planet',
                 text='Planet',
                 log_x=True
                 )  # Set logarithmic scale for x-axis

# change the text position i.e. planet names
fig.update_traces(textposition='bottom center', selector=dict(type='scatter'), textfont=dict(color='white'))
# change the size of the figure and its background color
fig.update_layout(width=1300, height=820, plot_bgcolor='black', legend=dict(bgcolor='darkgray', bordercolor='gray', borderwidth=1))
# adding horizontal grid lines
fig.update_yaxes(title_text='Surface Gravity(m/s2)', gridcolor='grey', zerolinecolor='grey')
# making x-axis invisible
fig.update_layout(xaxis=dict(visible=True),template='plotly_dark')
# data to be displayed when hovering over the planet for allplanets slider
fig.update_traces(hovertemplate="""<b>Planet: %{text}</b><br>
                  <br><span style='font-size: 12px; font-family: Sans-serif'>Surface Gravity(m/s2): %{y:.2f} g/cm³</span>
                  <br><span style='font-size: 12px; font-family: Sans-serif'>Volume(km3): %{customdata[0]} X 10<sup>%{customdata[1]}</sup></span>
                  <br><span style='font-size: 12px; font-family: Sans-serif'>Orbit Distance(km) :%{customdata[4]}</span>
                  <extra></extra>""")


# Adding background image
fig.update_layout(images=[dict(
    source="https://media.istockphoto.com/id/1372399516/vector/dark-blue-defocused-blurred-motion-gradient-soft-abstract-background-vector.jpg?s=612x612&w=0&k=20&c=ulLw1_awuztJ1ujhXSBAjQpI4L5PHuF-RE01zlAQHZ0=",
    xref="paper", yref="paper",
    x=0, y=1,
    sizex=1, sizey=1,
    sizing="stretch",
    opacity=0.5,
    layer="below"
)])
# Set the background color to be black
fig.update_layout(legend=dict(bgcolor='black', bordercolor='black', borderwidth=1))
# Everytime we hover over the planet the following details will be displayed
def get_hover_template(planet):
    if planet == "All Planets":
        return "<b>Planet: %{text}</b><br><br>" \
               "Surface Gravity(m/s2): %{y:.2f} g/cm3<br>" \
               "Volume(km3): %{customdata[0]} X 10<sup>%{customdata[1]}</sup><extra></extra>" \
               "<br>Orbit Distance(km) :%{customdata[4]}" \
               "<extra><extra>"
    else:
        return f"<b>Planet: {planet}</b><br>" \
               f"<br>Volume: {df[df['Planet'] == planet]['Mantissa_volume'].values[0]}" \
               f"X 10<sup>{df[df['Planet'] == planet]['Exponent_volume'].values[0]} </sup>" \
               f"<br>Mass: {df[df['Planet'] == planet]['Mantissa_mass'].values[0]}" \
               f"X 10<sup>{df[df['Planet'] == planet]['Exponent_mass'].values[0]} </sup>"\
               f"<br>Orbit Distance(km): {df1[df1['Planet'] == planet]['Orbit Distance(km)'].values[0]}" \
               f"<br>Surface Gravity: {df1[df1['Planet'] == planet]['Surface Gravity(m/s2)'].values[0]}"\
               f"<br>Density: {df1[df1['Planet'] == planet]['Density(g/cm3)'].values[0]}"\
               f"<br>Equatorial Radius(km): {df1[df1['Planet'] == planet]['Equatorial Radius(km)'].values[0]}"\
               f"<br>Atmospheric Constituents: {df1[df1['Planet'] == planet]['Atmospheric Constituents'].values[0]}"\
               f"<br>Mean Orbit Velocity(km/h): {df1[df1['Planet'] == planet]['Mean Orbit Velocity(km/h)'].values[0]}"\
               f"<br>Mean Orbit Distance(km): {df1[df1['Planet'] == planet]['Orbit Distance(km)'].values[0]}"\
               f"<br>Number of Moons: {df1[df1['Planet'] == planet]['Moons'].values[0]}"\
               f"<br>Do they have Rings?: {df1[df1['Planet'] == planet]['Rings'].values[0]}"\
               f"<extra></extra>"

# Create the dropdown menu
fig.update_layout(
    updatemenus=[
        {
            "active": 0,  # setting slider to 'All Planets' option initially
            "buttons": [
                {
                    "label": "Solar System",
                    "method": "update",
                    "args": [{"visible": True,
                              "hovertemplate": get_hover_template("All Planets")},
                             {"title": "Surface Gravity of each Solar System Planets",
                              "yaxis.visible": True}]
                },
                *[
                    {
                        "label": planet,
                        "method": "update",
                        "args": [{"visible": [planet == p for p in df['Planet']],
                                  "hovertemplate": get_hover_template(planet)},
                                 {"title": f"Planet: {planet}",
                                  "yaxis.visible": True}],
                        "name": planet
                    }
                    for planet in df['Planet']
                ]
            ],
            "x": 1.05,
            "y": 1.05,
            "showactive": True,
            "xanchor": "left",
            "yanchor": "top",
            'bgcolor':'#f2d8d8',
            "font": {"color": "black"}
        }
        # "#active": "gray"  # Color when hovering over dropdown items
    ]
)
# Update the y-axis label with units in cm³
fig.update_layout(yaxis_title='Surface Gravity(m/s2)')
# Update the y-axis label with units in cm³
fig.update_layout(
    xaxis_title='Orbit Distance from the Sun (Km)',
    xaxis=dict(
        title_standoff=30
    )
)
# modifying the font size and colors
fig.update_layout(
    title={
        'text': "Surface Gravity of each Solar System Planets",  # Specify the title text here
        'font': {
            'family': "Sans-serif",  # Specify the font family here
            'size': 30,         # Specify the font size here
            'color': "white"   # Specify the font color here
        },
        'x': 0.070,  # Center horizontally
        # 'y': 0.95,   # Vertical alignment
    },
    yaxis=dict(
        tickfont=dict(
            family="Sans-serif",  # Specify the font family for y-axis labels
            size=14,         # Specify the font size for y-axis labels
            color="white"    # Specify the font color for y-axis labels
        )
    ),
    legend=dict(
        font=dict(
            family="Sans-serif",  # Specify the font family for legends
            size=12,         # Specify the font size for legends
            color="white"    # Specify the font color for legends
        ),
        bgcolor='black'      # Specify the background color for legends
    )
)
# Set y-axis tickprefix to repeat the units
fig.update_yaxes(ticksuffix=' m/s2')
# set the planet labels font size
fig.update_traces(
    textposition='bottom center',
    selector=dict(type='scatter'),
    textfont=dict(color='white', size=12),  # Adjust the size parameter as needed
    showlegend=False        # removing the legend
)
# Add annotation for the description
fig.update_layout(
    annotations=[
        {
            "text": "Hover over the planets to learn more",
            "xref": "paper",
            "yref": "paper",
            "x": 0.001,
            "y": 1.05,
            "showarrow": False,
            "font": {
                "family": "Sans-serif",
                "size": 14,
                "color": "grey"
            }
        },
        {
            "text": "The x-axis scale represents values increasing by 10 million.",  # Description text
            "x": 0.001,  # X position (center)
            "y": -0.07,  # Y position (negative value to place it below the slider bar)
            "showarrow": False,  # Hide arrow
            "font": {"family":"Sans-serif","size": 12, "color": "grey"},
            "xref": "paper",
            "yref": "paper",
            "align": "left",  # Align text to the left
        }
    ]
)

fig.data[0].marker.line.color = "white"
fig.data[1].marker.line.color='yellow'
fig.data[2].marker.line.color='blue'
fig.data[3].marker.line.color = "red"

fig.show()

In [2]:
# SECOND VISUALISATION

import pandas as pd
import plotly.express as px

# Read data
rotation_velocity = pd.read_csv('Planet.csv', encoding='iso-8859-1')

# Remove commas from 'Mean Orbit Velocity(km/h)' column and convert to numeric
rotation_velocity['Mean Orbit Velocity(km/h)'] = rotation_velocity['Mean Orbit Velocity(km/h)'].str.replace(',', '')
rotation_velocity['Mean Orbit Velocity(km/h)'] = pd.to_numeric(rotation_velocity['Mean Orbit Velocity(km/h)'])

# Sort the DataFrame based on 'Mean Orbit Velocity(km/h)' column
rotation_velocity = rotation_velocity.sort_values(by='Mean Orbit Velocity(km/h)', ascending=False)

# Selecting sequential color for each planet
color_scale={
    'Mercury': '#A9A9A9',     # Silver Chalice
    'Venus': '#FFD700',       # Gold
    'Earth': '#4682B4',       # Steel Blue
    'Mars': '#CD5C5C',        # Chestnut Rose
    'Jupiter': '#FF6347',     # Persimmon
    'Saturn': '#ffdea7',      # Navajo White
    'Uranus': '#87CEEB',      # Seagull
    'Neptune': '#3d5ef9'      # Dodger Blue
}
fig = px.bar(rotation_velocity,
             x="Mean Orbit Velocity(km/h)",
             y="Planet",
             orientation='h',
             title="Mean Orbital Velocity (km/h) of Solar System Planets",
             labels={"Mean Orbit Velocity(km/h)": "Mean Orbit Velocity (km/h)"},
             template='plotly_dark',
             color_discrete_map=color_scale,  # Use color_discrete_map to assign colors based on color_map
             color="Planet")  # Use "Planet" column for color assignment

# Calculate the maximum value of the bars
max_value = rotation_velocity['Mean Orbit Velocity(km/h)'].max()
# Add annotations to each bar
for i in range(len(rotation_velocity)):
    orbit_velocity = rotation_velocity.iloc[i]['Mean Orbit Velocity(km/h)']
    orbit_velocity_k = "{:.2f}k".format(orbit_velocity / 1000)  # Format value in "k" format
    # Set x-coordinate as a fraction of the maximum value of the bars
    x_coordinate = max_value * 0.025
    fig.add_annotation(
        x=orbit_velocity + x_coordinate,
        y=rotation_velocity.iloc[i]['Planet'],
        text=orbit_velocity_k,  # Value to display in "k" format
        showarrow=False,
        font=dict(family="Sans-serif",color='white', size=12),
        align='left'
    )
# Customizing layout
fig.update_layout(
    width=1200,
    height=650,
    plot_bgcolor='rgba(0,0,0,0)',
    title_font_size=22,
    font=dict(
        family="Sans-serif",
        size=12,
        color="white"
    ),
    xaxis_title="Mean Orbit Velocity (km/h)",
    yaxis_title="Planet",
    xaxis_showgrid=True,
    yaxis_showgrid=False,
    bargap=0.3,
    hoverlabel=dict(font_size=12),
    showlegend=False  # Remove legend
)

fig.show()
